In [1]:
# ###################################
# Created by: Angy Sanchez (@ansanc)
# Date: 15/02/2024
# ###################################
# Importar librerias
# ###################################
# pandas para manejo de dataframes
# os para manejo de archivos excel
# numpy para operaciones de datos 
# geopandas para manejo de datos geograficos
# folium para manipulacion de mnapas y creacion de heatmaps

import pandas as pd
import numpy as np
import os 
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from folium.plugins import FloatImage
from folium.plugins import MarkerCluster
from folium.features import DivIcon
# Para tener formato local de dinero
import locale
# para quitar las tildes y  para manejo uniforme de los strings
from unidecode import unidecode
# para leer la API de datos.gov.co (Portal de datos colombianos)
import requests
import json
from sodapy import Socrata

In [ ]:
##################
# CONSTANTES
##################
locale.setlocale(locale.LC_ALL, 'en_CO.UTF-8')
#######################################################################
# Se define una función para usar unidecode para eliminar las tildes
#######################################################################
def remove_accents(s):
    return unidecode(s)

In [ ]:
# #################################
# INFORMACION df_daneENADAS - COLOMBIA
# #################################

# Informacion oficial del dane de los departamentos de colombia desde la API de datos.gov.co
client = Socrata("www.datos.gov.co", None)
dane = client.get("vafm-j2df", limit =1130) #DIVIPOLA CON CORDENADAS "vafm-j2df" #Divipola solo nombres "gdxc-w37w"
df_dane =  pd.DataFrame.from_records(dane)

# Las APIs no funcionan en ciertas redes de acuerdo a la seguraridad 
#por lo que en dado caso que no se puede descargar la informacion en excel desde la pagina de Divipola del DANE
#df_dane = pd.read_excel(r'DIVIPOLA_Municipios.xlsx')

############################################
# Limpieza de datos base divipola - DANE
############################################
# El archivo de divipola puede llegar a tener ciertas columnas inciales de texto que no son de interes
# Por lo que se eliminan estas filas y columnas que no son interes
df_dane.drop(columns=df_dane.columns[4],inplace=True)
df_dane.drop([0,1,2,3,4,5,6,7,8,9],inplace=True)
df_dane = df_dane[:-7]
df_dane.reset_index(drop=True, inplace = True)
# Se renombran las columnas como referencia de preferencia
df_dane.columns = ['DEPT_CODIGO', 'DEPARTAMENTO','MUNI_CODIGO','MUNICIPIO', 'LATITUD','LONGITUD']
# Se modifica el tipo de latitud y longitud para asegurarnos que sean tipo numero y no string
df_dane['LATITUD']= df_dane['LATITUD'].astype(float)
df_dane['LONGITUD']= df_dane['LONGITUD'].astype(float)
# Dado a que los municipios pueden llegar a tener tildes y/o caracteres que pueda llegar a no ser comparable se unifica los formatos
df_dane['MUNICIPIO'] = df_dane['MUNICIPIO'].apply(remove_accents)
df_dane['MUNICIPIO'] = df_dane['MUNICIPIO'].str.title()
df_dane['DEPT'] = df_dane['DEPT'].apply(remove_accents)
df_dane['DEPT'] = df_dane['DEPT'].str.title()
# se crea un df con los municipios principales
df_munprin = df_dane.loc[df_dane['MUNICIPIO'].str.endswith('001')]

In [ ]:
#############
# MAPAS
#############
# Variable para ver si se desea los mapas con el texto en el mapa o como pop up
text = input('¿DESEA VER TEXTO SOBRE EL MAPA? ¿SI O NO?:').upper()

#######################################
# Mapa de los muncipios de Colombia
#######################################
# Informacion del mapa de colombia. 
# Se puede descargar del IGAC(https://geoportal.igac.gov.co/contenido/datos-abiertos-cartografia-y-geografia)
mapa_col = gpd.read_file("infoGeo\COLOMBIA\COLOMBIA.shp")

###############################
# MAPA GENERAL
###############################
# Creación de la base del mapa
# Algunas opciones para mapa base: "cartodb positron", "cartodbdark_matter"
m = folium.Map(location=[df_dane['LATITUD'].mean(), df_dane['LONGITUD'].mean()],
               max_bounds=True,
               zoom_start=10,
               tiles="cartodb positron")#None
# limites del mapa
folium.FitBounds([(df_dane['LATITUD'].mean()-5,df_dane['LONGITUD'].mean()-5), (df_dane['LATITUD'].mean()+5, df_dane['LONGITUD'].mean()+5)]).add_to(m)

# Agregar capa con los departamentos de Colombia
folium.GeoJson(data=mapa_col["geometry"],name = 'Colombia',show = False).add_to(m)

# Barra de buscador
#folium.plugins.Geocoder().add_to(m)
# Mapa en pantalla completa
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)
# Se incluyen e
for i in df_dane['MUNICIPIO']:
    reg = df_dane[df_dane['MUNICIPIO']==i] 
folium.CircleMarker(
    location=[reg['LATITUD'].mean(), reg['LONGITUD'].mean()],
    #radius=reg['PORCENTAJE_SALDO'].sum(),
    #color = colores_reg[i],
    #fill=colores_reg[i],
    #name = 'Porcentaje saldo (%)',
    name = 'Municipios de Colombia',
    #popup=folium.Popup("Porcentaje saldo : %s %%"%round(reg['PORCENTAJE_SALDO'].sum(),2))
    popup=folium.Popup(i),
).add_to(m)
if text == 'SI':
    folium.Marker(location = [reg['LATITUD'].mean() + 0.5, reg['LONGITUD'].mean() - 1.6],
            icon = DivIcon(
            icon_size=(100,30),
            icon_anchor=(-50,3),
            html='<div style="font-size: 14pt">%s'+i+' %%</div>'),
).add_to(m)
# Modificacion del titulo del mapa resultante
title_general = '''
             <h3 align="center" style="font-size:20px"><b>Municipios de Colombia</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_general))


# Disponibilización de capas
folium.LayerControl().add_to(m)

# Guardar el mapa como html 
m.save('map.html')

In [ ]:
##########################################################
# MAPA GENERAL DE LA UBICACION DE LOS CENTROS DE COSTOS
##########################################################
# Creación de la base del mapa
# Algunas opciones para mapa base: "cartodb positron", "cartodbdark_matter"
m = folium.Map(location=[dff['LATITUD'].mean(), dff['LONGITUD'].mean()],
               max_bounds=True,
               zoom_start=10,
               tiles=None)#"cartodb positron")#None
# limites del mapa
folium.FitBounds([(dff['LATITUD'].mean()-5,dff['LONGITUD'].mean()-5), (dff['LATITUD'].mean()+5, dff['LONGITUD'].mean()+5)]).add_to(m)

# Agregar capa con los departamentos de Colombia
folium.GeoJson(data=mapa_col["geometry"],name = 'Colombia',show = False,    style_function=lambda feature: {
        #"fillColor": "#ffff00",
        "color": "grey",
        #"weight": 2,
        #"dashArray": "5, 5",
    },).add_to(m)

# Barra de buscador
#folium.plugins.Geocoder().add_to(m)
# Mapa en pantalla completa
folium.plugins.Fullscreen(
    position="topright",
    title="Expand me",
    title_cancel="Exit me",
    force_separate_button=True,
).add_to(m)

for i in dff['BCECO_CODIGO']:
    reg = dff[dff['BCECO_CODIGO'] == i] 
    folium.CircleMarker(
        location=[reg['LATITUD'].mean(), reg['LONGITUD'].mean()],
        radius=0.5,#reg['PORCENTAJE_SALDO'].sum(),
        #color = colores_reg[i],
        #fill=colores_reg[i],
        name = 'Ubicacion centro de costos',
        popup = folium.Popup(folium.IFrame(reg[['BCECO_CODIGO','MUNICIPIO']].to_html(index=False),
                       width=100,
                       height=100),
                     max_width=100)
    ).add_to(m)
    #if text == 'SI':
    #    folium.Marker(location = [reg['LATITUD'].mean() + 0.5, reg['LONGITUD'].mean() - 1.6],
    #            icon = DivIcon(
    #            icon_size=(100,30),
    #            icon_anchor=(-50,3),
    #            html=reg[['BCECO_CODIGO','MUNICIPIO']].to_html(index=False))
    #            ).add_to(m)

# Modificacion del titulo del mapa resultante
title_general = '''
             <h3 align="center" style="font-size:20px"><b>Centros de costos - Banco de bogota</b></h3>
             '''
m.get_root().html.add_child(folium.Element(title_general))


# Disponibilización de capas
folium.LayerControl().add_to(m)

# Guardar el mapa como html 
m.save('map_ubicacioncc.html')


In [ ]:
# Si se quiere referenciar ciertas ubicacionas en vez de ver los municipios:
# Primero requerimos un dataframe con la informacion que queremos referencia
# Debemos tener:
# - Nombre o ID ref
# - Municipio
# - Departamento
# Se puede llegar a incluir una variable que pueda modificar el tamaño de nuestro putno en el mapa por ejemplo # personas o egresos